<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Settings" data-toc-modified-id="Settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Settings</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Digit-Classifier---KNN" data-toc-modified-id="Digit-Classifier---KNN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Digit Classifier - KNN</a></span></li></ul></div>

In [1]:
import sys
import os
import time
import datetime
import pickle
import json
import re
import operator
from collections import defaultdict

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
sys.path.append("/Users/xuzhu/Desktop/code/assistants") # my package

from toolbox.os_assistant import scan_folder

---

## Settings

In [3]:
data_folder = "/Users/xuzhu/Desktop/data/open_dataset"

# converted black img to 0,1 data already
project_folder = os.path.join(data_folder, "digits")
training_data_folder = os.path.join(project_folder, "training")
test_data_folder = os.path.join(project_folder, "test")

## Functions

In [4]:
def knn_classify(
    input_vector,
    trainig_set,
    labels,
    k
):
    """
    Here is the description
    """
    
    data_size = trainig_set.shape[0]
    diff_matrix = np.tile(input_vector, (data_size, 1)) - trainig_set # np.tile(input_array, repetitions)
    sq_diff_matrix = diff_matrix ** 2
    sq_distance = sq_diff_matrix.sum(axis=1)
        # >> sum(axis=1)
        # data1: (d1, d2, d3)  --> d1+d2+d3
        # data2: (d1, d2, d3)  --> d1+d2+d3
    distance = sq_distance ** 0.5
    sorted_distance_index_list = distance.argsort() # sort distance
    
    class_stats = {}
    for i in range(k):
        vote_label = labels[sorted_distance_index_list[i]]
        class_stats[vote_label] = class_stats.get(vote_label, 0) + 1
        
    sorted_class_stats = sorted(
        class_stats.items(), # an item: (label, value) --> index: 0 is label, 1 is value
        key=operator.itemgetter(1),
            # NOTE: The key parameter is always a function that is fed one item from the iterable
        reverse=True
    )
    
    label_output = sorted_class_stats[0][0] # [(label, value), (label, value), ...] 
        # choose the label with the max cnt value
    return label_output

In [5]:
# TEST
# test_filename = scan_results["file"]["txt"][0]
# test_filepath = os.path.join(training_set_folder, test_filename)
# test_filepath
# return_vector = np.zeros((1, 1024)) # 1024 = 32x32 img
# with open(test_filepath, "r") as f:
#     size = 32 
#     for i in range(size): # 32 rows
#         line_str = f.readline()
#         for j in range(size): # the first 32 characters
#             return_vector[0, 32*i+j] = line_str[j]
# return_vector[0, 11]

def convert_img_to_vector__temp(img_data_filepath):
    return_vector = np.zeros((1, 1024)) # 1024 = 32x32 img
    with open(img_data_filepath, "r") as f: # f.readlines()
        size = 32 
        for i in range(size): # 32 rows
            line_str = f.readline()
            for j in range(size): # read the first 32 characters
                return_vector[0, 32*i+j] = line_str[j]
                
    return return_vector

## Digit Classifier - KNN

In [7]:
training_data_folder_scan_results = scan_folder(training_data_folder)
training_filenames = training_data_folder_scan_results["file"]["txt"]
training_file_cnt = len(training_filenames)

# Training Data
training_matrix = np.zeros((training_file_cnt, 1024)) # 32x32 img
training_data_labels = []
for i in range(training_file_cnt):
    filename = training_filenames[i] # {label}_{No.}.txt
    label = int(filename.split("_")[0])
    training_data_labels.append(label)
    
    filepath = os.path.join(training_data_folder, filename)
    training_matrix[i, :] = convert_img_to_vector__temp(filepath)


        
============================== SCAN FOLDER ==============================
Folder Path: '/Users/xuzhu/Desktop/data/open_dataset/digits/training'
Subfolders: 0 => []
Filetypes: 1 => ['txt']
-'txt' file count: 1934


In [8]:
test_data_folder_scan_result = scan_folder(test_data_folder)
test_filenames = test_data_folder_scan_result["file"]["txt"]
test_file_cnt = len(test_filenames)

# Test Data
error_cnt = 0
for i in range(test_file_cnt):
    filename = test_filenames[i]
    label = int(filename.split("_")[0])
    
    filepath = os.path.join(training_data_folder, filename)
    vector_under_test = convert_img_to_vector__temp(filepath)
    test_result = knn_classify(
        input_vector=vector_under_test,
        trainig_set=training_matrix,
        labels=training_data_labels,
        k=8
    )
    # print("The KNN classifier return with {0}; the real label is {1}".format(test_result, label))
    
    if test_result != label:
        error_cnt += 1

print("\n\n{0} Digit Classifier Outputs(KNN) {1}".format("="*30, "="*30))
print("The total number of test cases is {0}".format(test_file_cnt))
print("The total number of errors is {0}".format(error_cnt))
print("The total error rate is {0}"\
      .format(
        str(round(100*error_cnt/test_file_cnt, 2))+"%"
        )
    )


        
============================== SCAN FOLDER ==============================
Folder Path: '/Users/xuzhu/Desktop/data/open_dataset/digits/test'
Subfolders: 0 => []
Filetypes: 1 => ['txt']
-'txt' file count: 946


============================== Digit Classifier Outputs(KNN) ==============================
The total number of test cases is 946
The total number of errors is 18
The total error rate is 1.9%
